In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Read the saved data
print("=" * 70)
print("READING SAVED DATA")
print("=" * 70)

try:
    df = pd.read_csv('students_dataset.csv')
    print(f"✅ Dataset loaded successfully!")
    print(f" Shape: {df.shape}")
    print(f" Columns: {len(df.columns)}")
    print(f" Total records: {len(df)}")
except FileNotFoundError:
    print(" File 'students_dataset.csv' not found.")
    exit()



In [ ]:
# 1. Check the basic structure
print("="*60)
print("DATA STRUCTURE ANALYSIS")
print("="*60)

print(f"Dataset Shape: {df.shape}")
print(f"Number of students: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 2. View column categories
print("\n" + "="*60)
print("COLUMN CATEGORIES")
print("="*60)

# Group columns by type
test_score_cols = [col for col in df.columns if 'Test_Score' in col]
attendance_cols = [col for col in df.columns if 'Attendance' in col]
homework_cols = [col for col in df.columns if 'Homework' in col]
participation_cols = [col for col in df.columns if 'Participation' in col]
textbook_cols = [col for col in df.columns if 'Textbook' in col]

print(f"Test Score columns: {len(test_score_cols)}")
print(f"Attendance columns: {len(attendance_cols)}")
print(f"Homework columns: {len(homework_cols)}")
print(f"Participation columns: {len(participation_cols)}")
print(f"Textbook columns: {len(textbook_cols)}")

# 3. Check subject coverage by grade
print("\n" + "="*60)
print("SUBJECT COVERAGE BY GRADE")
print("="*60)

for grade in range(1, 13):
    grade_test_cols = [col for col in test_score_cols if f'Grade_{grade}_' in col]
    if grade_test_cols:
        subjects = list(set([col.split(f'Grade_{grade}_')[1].split('_Test')[0]
                           for col in grade_test_cols]))
        print(f"Grade {grade}: {len(grade_test_cols)} subjects - {subjects}")

all_categorized_cols = set(test_score_cols + attendance_cols + homework_cols + participation_cols + textbook_cols)
remaining_cols = [col for col in df.columns if col not in all_categorized_cols]

print("\n" + "="*60)
print("REMAINING COLUMNS")
print("="*60)
print(f"Number of remaining columns: {len(remaining_cols)}")
print("Remaining columns (first 20):\n", remaining_cols[:20])
print("Remaining columns (last 20):\n", remaining_cols[-20:])

In [ ]:
# Check for nulls per column
print("==============================")
print("CHECKING FOR MISSING DATA")
print("==============================")
print(df.isnull().sum())

# Check the percentage of missing data
print("================================")
print("Percentage of missing data:")
print("--------------------------------")
print(df.isnull().mean() * 100)

In [ ]:
print("--------------------------------")
print("Check data types of all columns")
print("--------------------------------")
print(df.dtypes)

# To see a count of how many columns you have for each type
print(df.dtypes.value_counts())

# Select only numeric columns (float and int)
numeric_cols = df.select_dtypes(include=['number']).columns
print(f"Numeric Columns: {list(numeric_cols)}")

# Select only categorical/object columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
print(f"Categorical Columns: {list(categorical_cols)}")

#column name of one subject(maths)
one_column= [col for col in df.columns if 'Grade_12' in col and 'Math' in col]
#number of column is one subject (maths)
print("number of column in one subject is:",len(one_column))
print(one_column)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Get the value counts of dtypes
dtype_counts = df.dtypes.value_counts().reset_index()
dtype_counts.columns = ['Data Type', 'Count']

# 2. Plotting with the fix
plt.figure(figsize=(10, 6))

# Fix: Assign 'Data Type' to hue and set legend=False
sns.barplot(
    data=dtype_counts,
    x='Data Type',
    y='Count',
    hue='Data Type',
    palette='viridis',
    legend=False
)

plt.title('Distribution of Data Types in Student Dataset', fontsize=14)
plt.ylabel('Number of Columns')
plt.xlabel('Data Type')

# 3. Add labels on top of bars
for i, count in enumerate(dtype_counts['Count']):
    plt.text(i, count + 5, str(count), ha='center', fontweight='bold')

plt.show()

In [ ]:
# 1. Check for inconsistent scales
print("\n" + "="*60)
print("SCALE CONSISTENCY CHECK")
print("="*60)

# Test scores should be on similar scale (0-100 typically)
sample_test_scores = test_score_cols[:10]  # Check first 10 test score columns

for col in sample_test_scores:
    if col in df.columns:
        min_val = df[col].min()
        max_val = df[col].max()
        mean_val = df[col].mean()
        print(f"{col:50} Min: {min_val:6.2f} Max: {max_val:6.2f} Mean: {mean_val:6.2f}")

# 2. Check for impossible values
print("\n" + "="*60)
print("DATA VALIDATION CHECKS")
print("="*60)

# Check if any test scores are outside reasonable range (0-100)
for grade in [9, 10, 11, 12]:  # Check upper grades first
    grade_cols = [col for col in test_score_cols if f'Grade_{grade}_' in col]
    for col in grade_cols[:3]:  # Check first 3 subjects per grade
        if col in df.columns:
            invalid_count = ((df[col] < 0) | (df[col] > 100)).sum()
            if invalid_count > 0:
                print(f"WARNING: {col} has {invalid_count} values outside 0-100 range")

# 3. Check attendance, homework, participation scales
# These are likely percentages (0-100) or proportions (0-1)
print("\nChecking behavioral data scales...")
sample_attendance = attendance_cols[0] if attendance_cols else None
if sample_attendance and sample_attendance in df.columns:
    att_min = df[sample_attendance].min()
    att_max = df[sample_attendance].max()
    print(f"Attendance sample ({sample_attendance}): {att_min:.2f} to {att_max:.2f}")

    # Determine scale
    if att_max > 1:
        print("Likely scale: 0-100 (percentage)")
        # We may need to normalize to 0-1 for consistency
    else:
        print("Likely scale: 0-1 (proportion)")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# Distribution of key variables
# -----------------------------
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Distribution of Total_National_Exam_Score
if 'Total_National_Exam_Score' in df.columns:
    axes[0, 0].hist(
        df['Total_National_Exam_Score'].dropna(),
        bins=30,
        edgecolor='black',
        alpha=0.7
    )
    axes[0, 0].set_title('Distribution of Total_National_Exam_Score')
    axes[0, 0].set_xlabel('Average Score')
    axes[0, 0].set_ylabel('Frequency')

# Plot 2: Parental Involvement distribution
if 'Parental_Involvement' in df.columns:
    axes[0, 1].hist(
        df['Parental_Involvement'].dropna(),
        bins=30,
        edgecolor='black',
        alpha=0.7,
        color='green'
    )
    axes[0, 1].set_title('Parental Involvement Distribution')
    axes[0, 1].set_xlabel('Involvement Score (0–1)')
    axes[0, 1].set_ylabel('Frequency')

# Plot 3: Overall Average by Academic Track (BOXPLOT)
if 'Field_Choice' in df.columns and 'Overall_Average' in df.columns:
    track_data = df[['Field_Choice', 'Overall_Average']].dropna()

    tracks = sorted(track_data['Field_Choice'].unique())
    labels = ['Social' if t == 0 else 'Natural' for t in tracks]

    box_data = [
        track_data[track_data['Field_Choice'] == t]['Overall_Average']
        for t in tracks
    ]

    axes[1, 0].boxplot(
        box_data,
        tick_labels=labels   # ✅ FIXED (was labels=)
    )
    axes[1, 0].set_title('Overall Average by Academic Track')
    axes[1, 0].set_ylabel('Average Score')

# Plot 4: Region performance
if 'Region' in df.columns and 'Overall_Average' in df.columns:
    region_avg = (
        df.groupby('Region')['Overall_Average']
        .mean()
        .sort_values()
    )

    axes[1, 1].barh(range(len(region_avg)), region_avg.values)
    axes[1, 1].set_yticks(range(len(region_avg)))
    axes[1, 1].set_yticklabels(region_avg.index)
    axes[1, 1].set_title('Average Performance by Region')
    axes[1, 1].set_xlabel('Average Score')

plt.tight_layout()
plt.show()

In [ ]:
#Drop ID (never use in ML)
df = df.drop(columns=['Student_ID'], errors='ignore')

#  Encode Field_Choice (VERY IMPORTANT)
df['Field_Choice'] = df['Field_Choice'].map({
    'Social': 0,
    'Natural': 1
})

# HANDLE CAREER_INTEREST
# Fill missing with "Unknown"
df['Career_Interest'] = df['Career_Interest'].fillna('Unknown')